In [28]:
#my CDF client uses tokens to authenticate, and I have configured the setup in another file.
%run config.py


/Users/noahkarsky/.pyenv/versions/3.9.6/lib/python3.9/site-packages/cognite/client/_cognite_client.py:80: UserWarning: You are using version 2.31.0 of the SDK, however version 2.38.3 is available. Upgrade or set the environment variable 'COGNITE_DISABLE_PYPI_VERSION_CHECK' to suppress this warning.
  self._config = ClientConfig(


In [3]:
from cognite.client import CogniteClient
from cognite.client.data_classes import Sequence

In [57]:
#adapted from https://github.com/cognitedata/LAS-DLIS-to-Sequence

"""
Wrapper to lasio - extracts data from a las file into the expected CDF sequence format
"""
import lasio
from parse_files import *


def parse_las(filename, std_dict, nan):
    """
    Extract local LAS file contents to a dict with columns, data and metadata
    """
    las = las_read(filename)
    try:
        return {
            "columns": get_las_columns(las, std_dict),
            "data": get_las_data(las, nan),
            "metadata": get_las_metadata(las)
        }
    except Exception as err:
        raise FileConvertError(err)


def las_read(filename):
    """
    Read local LAS file and return its contents as a LASFile object
    """
    try:
        file = open(filename, 'r').read()
        return lasio.read(file, ignore_header_errors=True)
    except Exception as err:
        raise FileReadError(err)


def get_las_columns(las_log, std_dict):
    """
    Get curve information with standard name.
    The curve data is stored column-wised. 
    If there are duplicate curves(same mnemonics), add "_dup" to the name of the latter one(s).
    Returns a list of columns that match the Sequence requirements,
    and a list of curve names which will be used when inserting data.
    """
    columns = []
    names = []
    for curve in las_log.curves:
        basename = curve.mnemonic.upper()
        curve_name = std_dict.get(basename, basename)
        count = names.count(curve_name)
        names.append(curve_name)
        columns.append(
            {
                "name": curve_name,
                "externalId": curve_name if count == 0 else curve_name + "_" + str(count),
                "description": curve.descr,
                "valueType": "DOUBLE",
                "metadata": {"unit": curve.unit},
            }
        )
    return columns


def get_las_data(las_log, nan):
    """
    Get data from the LASFile object as rows.
    Returns a dictionary with row index as key and row data as value, which satisfy the Sequence requirements.
    """
    data = las_log.data
    rows = {}
    for index, row in enumerate(data):
        row = [nan if is_null(element) else float(element) for element in row]
        rows[index + 1] = row

    return rows


def get_las_metadata(las_log):
    """
    Returns a dictionary containing well information from the LASFile object.
    """
    metadata = {}
    for key in las_log.well.keys():
        metadata[key] = "unit : {}, value : {}, descr : {}".format(
            las_log.well[key].unit, las_log.well[key].value, las_log.well[key].descr
        )
    return metadata

def find_assets(name,):
    f = c.assets.search(filter={"name": name, })
    return f[0] if f and len(f) > 0 else None


def find_potential_assets(name, type_name):
    f = find_assets(name, type_name)
    if f is None:
        similar_name1 = rename_well(name)
        f = find_assets(similar_name1, type_name)
    if f is None:
        similar_name2 = similar_name1.replace("_", "/")
        f = find_assets(similar_name2, type_name)
    if f is None:
        similar_name3 = similar_name1.replace(" ", "-")
        f = find_assets(similar_name3, type_name)
    return f


def create_sequence(file_desc, columns, rows, metadata):
    wbname = file_desc.get("wbname")
    ext_id = file_desc.get('ext_id')
    assetid = file_desc.get("assetId")
    datatype = file_desc.get("datatype") + file_desc.get("subtype", "")
    filename = file_desc.get("filename")
    filetype = filename.split(".")[-1]
    metadata.update({
        "type": "Logs",
        "wellbore_name": wbname,
        "filename": filename,
        "filetype": filetype
    })
    description = "logs for wellbore " + file_desc.get("wbname")
    if c.sequences.retrieve(external_id=ext_id):
        print("Sequence {} already exists in CDF".format(ext_id))
        return False
    # Create the sequence definition and columns
    try:
        c.sequences.create(
            Sequence(
                name=wbname,
                asset_id=assetid,
                description=description,
                external_id=ext_id,
                metadata=metadata,
                columns=columns
            )
        )
    except Exception as e:
        print("Unable to create sequence from {}. Reason:".format(filename))
        print(e)
        return False
    # Post the data rows to the sequence
    try:
        c.sequences.data.insert(
            external_id=ext_id,
            column_external_ids=[c.get("externalId") for c in columns],
            rows=rows,
        )
    except Exception as e:
        print("Unable to post data to sequence {}. Reason:".format(filename))
        print(e)
        return False



In [55]:
las_data ={}
data =parse_las('utah_las_files/78-32/UOU_FORGE-78B-32_R1B_TD_SECTION_ConPr_R06_L004Up_QAIT_QSLT_QCNT_QTGC_MAIN-GenericV12.las', las_data, 0)

In [59]:
folder_list = ['utah_las_files/78-32','utah_las_files/58-32']

for folder in folder_list:
    asset_external_id = folder.split('/')[-1]
    asset = find_assets(asset_external_id)
    for file in os.listdir(folder):
        if file.endswith(".las"):
            print(file)
            file_desc = {}
            file_desc['wbname'] = asset_external_id
            file_desc['ext_id'] = 'log-' + asset_external_id
            file_desc['datatype'] = 'las'
            file_desc['assetId'] = asset.id
            file_desc['filename'] = file
            las_data={}
            data = parse_las(folder + '/' + file, las_data, -999.25)
            create_sequence(file_desc, data['columns'], data['data'], data['metadata'])
    


UOU_FORGE-78B-32_R1B_TD_SECTION_ConPr_R06_L004Up_QAIT_QSLT_QCNT_QTGC_MAIN-GenericV12.las
UOU_FORGE-78B-32_R1B_5.75in_QSLT_SONIC_PnS_7495-9540ft_LAS.las
Sequence log-78-32 already exists in CDF
UOU_FORGE-78B-32_R1B_TD_SECTION_ConPr_R06_L003Down___QAIT_QSLT_QCNT_QTGC-GenericV12.las
Sequence log-78-32 already exists in CDF
UOU_FORGE-78B-32_R2A_HLDS-QCNT-QAIT_Repeat Pass-GenericV12.las
Sequence log-78-32 already exists in CDF
UOU_FORGE-78B-32_R1B_TD_SECTION_ConPr_R06_L001Down___QAIT_QSLT_QCNT_QTGC-GenericV12.las
Sequence log-78-32 already exists in CDF
UOU_FORGE-78B-32_R1B_TD_SECTION_ConPr_R06_L002Up___QAIT_QSLT_QCNT_QTGC_REPEAT-GenericV12.las
Sequence log-78-32 already exists in CDF
UOU_FORGE-78B-32_R2A_HLDS-QCNT-QAIT_Main Pass-GenericV12.las
Sequence log-78-32 already exists in CDF
58-32_main.las
58-32_sonic.las
Sequence log-58-32 already exists in CDF
Acord 1-26_digitized_geophysical_well_log.las
Sequence log-58-32 already exists in CDF
